In [17]:
from amigocloud import AmigoCloud

from shapely import wkb
import geopandas
import pandas as pd

import ipywidgets as widgets
from datetime import datetime

In [18]:
acceso = open('accesos/api.txt','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [97]:

# funcion para conversion de wkb a poligonos
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

def get_canhero_prop(codigo):
    # consulta sql para extraer los lotes de un credito
    query = {'query': 'select codca cod_ca, ca__ero nombre, cod_prop, propiedad, round(sum(superficie)::numeric, 2) superf from dataset_287438 where codca=\'{cod}\' group by codca, ca__ero, cod_prop, propiedad'.format(cod=codigo)}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30232/sql', query)
    data = select['data']
    
    # carga de datos seleccionados a DadaFrame
    df = pd.DataFrame(data)
    df['cod_prop'] = df['cod_prop'].apply(int)
    # DataFrame a GeoDataFrame
    # data = geopandas.GeoDataFrame(df, geometry='geometria')
    # convercion a FeatureColleccion
    # ee_lotes = ee.FeatureCollection(data.__geo_interface__)
    return df

def get_prop(codigo):
    # consulta sql para extraer los lotes de un credito
    query = {'query': 'select cod_prop, propiedad, codca, ca__ero, round(sum(superficie)::numeric, 2) from dataset_287438 where cod_prop={cod} group by codca, ca__ero, cod_prop, propiedad'.format(cod=codigo)}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30232/sql', query)
    data = select['data']
    
    # carga de datos seleccionados a DadaFrame
    df = pd.DataFrame(data)
    #df['cod_prop'] = df['cod_prop'].apply(int)
    # DataFrame a GeoDataFrame
    # data = geopandas.GeoDataFrame(df, geometry='geometria')
    # convercion a FeatureColleccion
    # ee_lotes = ee.FeatureCollection(data.__geo_interface__)
    return df

out_props = widgets.Output(layout={'border': '1px solid black'})

In [103]:
COD_CA = 75


out_props.clear_output()
canero = get_canhero_prop(COD_CA)
lista_cods = canero['cod_prop'].tolist()

lista=[]
for i in lista_cods:
    res = get_prop(i)
    with out_props:
        display(res)

canero

,cod_ca,nombre,cod_prop,propiedad,superf
0,75,AGUILERA TARADELLES NELSON MARIANO,324,EL NARANJAL--AGUILERA,39.30
1,75,AGUILERA TARADELLES NELSON MARIANO,216,LA ESTACADA--AGUILERA,123.11
2,75,AGUILERA TARADELLES NELSON MARIANO,217,FLAMBOYAN--AGUILERA,80.23
3,75,AGUILERA TARADELLES NELSON MARIANO,256,LOS ANDES--AGUILERA,82.42
4,75,AGUILERA TARADELLES NELSON MARIANO,551,VILLA CAROLA,265.92


# PROPIEDADES

In [100]:
out_props

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'display_data', 'data': {'text/plain'…